In [75]:
%reload_ext lab_black
import pandas as pd
import numpy as np
from plotnine import *

In [103]:
# read data
od = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/coneel/10_code/od_deaths_state.csv?token=ARFW6V3OWJEBEXR6EAMOD4S7SS5MM"
)

population = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/ssloate/20_intermediate_files/countypopulations_clean.csv?token=ARFW6VZYS6HL4RX22US75US7S52K6"
)

In [104]:
# rename od columns to match
od = od.rename(columns={"County Code": "FIPS"})

# drop _merge column in popualtion dataset
population = population.drop(columns="_merge")

In [105]:
# remove Alaska from od Data
od = od[od["State"] != "AK"]

In [117]:
# merge dataset. Use right merge, because it's okay if some years/counties don't have data in the od dataset. That just means there were less than 10 deaths
od_pop = pd.merge(population, od, on=["FIPS", "Year"], how="right", indicator=True)

assert od_pop[od_pop["_merge"] == "both"].all

In [118]:
# convert years to integers
od_pop["Year"] = od_pop["Year"].astype("int")
od_pop.sample(25)

,Unnamed: 0,State Name,County_x,Year,Population,FIPS,State Abbr,County_y,State,Deaths,_merge
6906,37072.0,New York,Suffolk County,2012,1496982.0,36103,NY,Suffolk County,NY,287.0,both
656,50348.0,Texas,Brazos County,2008,185098.0,48041,TX,Brazos County,TX,10.0,both
650,50330.0,Texas,Brazoria County,2010,314428.0,48039,TX,Brazoria County,TX,24.0,both
6811,22766.0,Louisiana,St. Tammany Parish,2006,223133.0,22103,LA,St. Tammany Parish,LA,64.0,both
589,29327.0,Missouri,Boone County,2007,155666.0,29019,MO,Boone County,MO,15.0,both
457,59372.0,West Virginia,Berkeley County,2012,106965.0,54003,WV,Berkeley County,WV,27.0,both
1378,40572.0,Ohio,Clark County,2012,137171.0,39023,OH,Clark County,OH,36.0,both
5804,60131.0,West Virginia,Putnam County,2011,56030.0,54079,WV,Putnam County,WV,12.0,both
7182,37152.0,New York,Ulster County,2012,181538.0,36111,NY,Ulster County,NY,12.0,both
2058,33114.0,Nebraska,Douglas County,2014,543005.0,31055,NE,Douglas County,NE,48.0,both


In [119]:
# drop unneeded columns

od_pop = od_pop.drop(columns=["County_y", "State", "_merge"])

In [120]:
# rename columns for easier understanding
od_pop = od_pop.rename(
    columns={"State Name": "State", "County_x": "County"},
    errors="raise",
)

# reorder columns
od_pop = od_pop[
    ["FIPS", "State Abbr", "State", "County", "Year", "Deaths", "Population"]
]

# sort by state, county
od_pop = od_pop.sort_values(["State", "County", "Year"])
od_pop.sample(25)

,FIPS,State Abbr,State,County,Year,Deaths,Population
62,37001,NC,North Carolina,Alamance County,2006,10.0,141462.0
930,37027,NC,North Carolina,Caldwell County,2015,27.0,81518.0
1689,23005,ME,Maine,Cumberland County,2007,31.0,278781.0
2339,21067,KY,Kentucky,Fayette County,2011,41.0,301696.0
6145,6067,CA,California,Sacramento County,2011,232.0,1433730.0
5888,20155,KS,Kansas,Reno County,2014,10.0,63701.0
5361,26139,MI,Michigan,Ottawa County,2008,10.0,261906.0
774,42017,PA,Pennsylvania,Bucks County,2012,125.0,625346.0
5288,12095,FL,Florida,Orange County,2010,149.0,1148564.0
4171,39095,OH,Ohio,Lucas County,2007,84.0,445482.0


In [121]:
# check for null values
od_pop[od_pop["State"].isnull()]

,FIPS,State Abbr,State,County,Year,Deaths,Population
373,51515,NaN,NaN,NaN,2015,0.0,NaN
1484,51560,NaN,NaN,NaN,2015,0.0,NaN


In [125]:
# FIPS 51515 and 51560 represent two counties in Virginia, Bedford City and Clifton Forge Valley, that were incorporated in to other counties. It is safe to drop them

od_pop = od_pop[(od_pop["FIPS"] != 51515) & (od_pop["FIPS"] != 51560)]
od_pop[od_pop["State"].isnull()]

,FIPS,State Abbr,State,County,Year,Deaths,Population


In [126]:
# Create per capita deaths column
od_pop["Deaths Per 100,000 People"] = (od_pop["Deaths"] / od_pop["Population"]) * 100000

In [128]:
# save to excel

od_pop.to_csv(
    "/Users/samsloate/Desktop/Data_Science/Opioids_Project/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/death_data_with_pop.csv"
)